In [12]:
import numpy as np
from os.path import join
import rasterio as rio
import uavsar_pytools
import sys
sys.path.append('/Users/zachkeskinen/Documents/uavsar_snow/functions')
from tiff_convert import grd_tiff_convert

In [9]:
in_fp = '/Users/zachkeskinen/Documents/uavsar_snow/jemez_slc/alamos_35915_01_BU_s1_2x8.llh'
ann_fp = '/Users/zachkeskinen/Documents/uavsar_snow/jemez_slc/alamos_35915_20005_003_200212_L090HH_01_BU.ann'
out_fp = join(in_fp, '.tiff')
grd_tiff_convert(in_fp, out_fp, ann_fp)

KeyError: 'llh.set_rows'

In [7]:
?grd_tiff_convert

Signature: grd_tiff_convert(in_fp, out_dir, ann_fp=None, overwrite='user')
Docstring:
Converts a single binary image either polsar or insar to geotiff.
See: https://uavsar.jpl.nasa.gov/science/documents/polsar-format.html for polsar
and: https://uavsar.jpl.nasa.gov/science/documents/rpi-format.html for insar
and: https://uavsar.jpl.nasa.gov/science/documents/stack-format.html for SLC stacks.

Args:
    in_fp (string): path to input binary file
    out_dir (string): directory to save geotiff in
    ann_fp (string): path to UAVSAR annotation file
File:      ~/miniconda3/envs/uavsar/lib/python3.9/site-packages/uavsar_pytools/convert/tiff_conversion.py
Type:      function
